In [1]:
import requests
import json

In [2]:
def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    #response = requests.post(automat_url,json=message['message'])
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    #print(response.json())
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def ontology(message):
    url = 'https://stars-app.renci.org/sparql-kp/query'
    return post('ontology',url,message)
 
def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/graph',{'message':strider_answer})
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',{'message': coalesced_answer})
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',{'message': omni_answer})
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',{'message': weighted_answer})
    return scored_answer

def aragorn(message, coalesce_type):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

In [15]:
subq={
    "message": {
        "query_graph": {
            "nodes": {
                "a": {
                    "id": "HP:0040301"
                },
                "b": {
                    "category": "biolink:PhenotypicFeature"
                }
            },
            "edges": {
                "ab": {
                    "subject": "a",
                    "object": "b",
                    "predicate": "biolink:subclass_of"
                }
            }
        }}}

In [16]:
r = ontology(subq)

In [17]:
r

{'message': {'query_graph': {'nodes': {'a': {'id': 'HP:0040301'},
    'b': {'category': 'biolink:PhenotypicFeature'}},
   'edges': {'ab': {'predicate': 'biolink:subclass_of',
     'subject': 'a',
     'object': 'b'}}},
  'knowledge_graph': {'nodes': {'HP:0000079': {'name': 'Abnormality of the urinary system',
     'category': ['biolink:PhenotypicFeature']},
    'HP:0033354': {'name': 'Abnormal urine metabolite level',
     'category': ['biolink:PhenotypicFeature']},
    'HP:0031979': {'name': 'Abnormal urine carbohydrate level',
     'category': ['biolink:PhenotypicFeature']},
    'HP:0001939': {'name': 'Abnormality of metabolism/homeostasis',
     'category': ['biolink:PhenotypicFeature']},
    'HP:0011277': {'name': 'Abnormality of the urinary system physiology',
     'category': ['biolink:PhenotypicFeature']},
    'HP:0000119': {'name': 'Abnormality of the genitourinary system',
     'category': ['biolink:PhenotypicFeature']},
    'UPHENO:0001001': {'name': 'phenotype',
     'catego

In [10]:
rs = strider(subq)

In [11]:
rs

{'message': {'query_graph': {'nodes': {'b': {'id': None,
     'category': ['biolink:PhenotpyicFeature'],
     'is_set': False},
    'a': {'id': ['HP:0040301'], 'category': None, 'is_set': False}},
   'edges': {'ab': {'subject': 'a',
     'object': 'b',
     'predicate': ['biolink:subclass_of'],
     'relation': None}}},
  'knowledge_graph': {'nodes': {}, 'edges': {}},
  'results': []},
 'logs': [{'timestamp': '2021-05-12T19:41:24.955752',
   'level': 'ERROR',
   'code': None,
   'message': '{"message": "Could not find a plan to traverse query graph", "line_number": 157}'},
  {'timestamp': '2021-05-12T19:41:24.953549',
   'level': 'WARNING',
   'code': None,
   'message': '{"message": "\\n                We couldn\'t find any possible plans starting from a pinned node\\n                that traverse every edge and node in your query graph\\n            ", "line_number": 566}'},
  {'timestamp': '2021-05-12T19:41:24.951470',
   'level': 'INFO',
   'code': None,
   'message': '{"message": 